### Importation des bibliothèques

In [2]:
import numpy as np
import numpy.fft
import pandas as pd
import glob, os, json
import librosa
import librosa.display
import IPython.display as ipd
import matplotlib.pyplot as plt
import seaborn as sn
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import confusion_matrix


import soundfile as sf
import subprocess
import scipy.io.wavfile as wavfile

# A la fin, ne pas oublier de retirer les bibliothèques qu'on n'utilise pas

### Création df avec données et liste des fichiers audio

In [3]:
data_directory = 'archive'

json_pattern = os.path.join(data_directory, '*.json')
json_file_list = glob.glob(json_pattern)

wav_pattern = os.path.join(data_directory, '*.wav')
wav_file_list = glob.glob(wav_pattern)

dfs = []
for file in json_file_list:
    with open(file) as f:
        json_data = pd.json_normalize(json.loads(f.read()))
        json_data['file_name'] = file.rsplit("/", 1)[-1]
    dfs.append(json_data)
df0 = pd.concat(dfs)

In [4]:
df = df0

In [5]:
df = df.reset_index()
df.drop('index', inplace=True, axis=1)

In [6]:
df = df.dropna(subset=['status'])
df = df.dropna(subset=['age'])
df.drop(df[df.gender == 'other'].index, inplace=True)

df = df.head(1000)


In [7]:
Fe = 22050
df["file_name"] = df["file_name"].apply(lambda row: os.path.join(data_directory, row)[:-4]+'wav')
df["quantités d'énergie"] = df["file_name"].apply(lambda row: librosa.load(row)[0])
df["dft"] = df["quantités d'énergie"].apply(lambda row: np.fft.fft(row, 10000))
df["freq_ft"] = df["dft"].apply(lambda row: np.fft.fftfreq(len(row), 1/Fe))





/home/cytech/.local/lib/python3.8/site-packages/librosa/util/decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)
/home/cytech/.local/lib/python3.8/site-packages/librosa/util/decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)


In [8]:
df = df.reset_index()
df.drop('index', inplace=True, axis=1)

In [9]:

fondamentales = []
for i in df.index:
    f = df.iloc[i].at["freq_ft"][np.where(df.iloc[i].at["dft"] == max(df.iloc[i].at["dft"]))[0][0]]
    fondamentales.append(f)
mu, sigma = np.mean(fondamentales), np.std(fondamentales)
fondamentales = [(x-mu)/sigma for x in fondamentales]
df["fondamentale"] = fondamentales

df.loc[(df.status == 'COVID-19'),'status']= 0
df.loc[(df.status == 'symptomatic'),'status']= 0.5
df.loc[(df.status == 'healthy'),'status']= 1


In [10]:
L = [float(x) for x in df["age"].tolist()]
moy, sigma = np.mean(L), np.std(L)
ages = [(x-moy)/sigma for x in L]
df["age"] = ages

In [11]:
df.loc[(df.gender == 'male'),'gender']= 0.
df.loc[(df.gender == 'female'),'gender']= 1.


In [12]:
MNIST_train = df
print("Dimensions MNIST_train :", MNIST_train.shape)
X = MNIST_train.loc[:, ['age', 'gender', 'fondamentale']]
y = MNIST_train.loc[:, 'status'] 

Dimensions MNIST_train : (1000, 54)


In [13]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [14]:
X_train

,age,gender,fondamentale
502,0.754624,0.0,-3.448375
645,-0.417817,0.0,0.351881
332,-0.141948,1.0,-0.332091
689,0.892559,0.0,-2.186992
139,-0.210915,0.0,0.045829
...,...,...,...
389,-0.486784,1.0,0.048307
119,-1.866126,0.0,0.356837
230,-0.762652,0.0,-0.107817
89,-1.107488,0.0,-0.107817


In [65]:
y_train

502      1
645    0.5
332      1
689      1
139    0.5
      ... 
389      0
119      1
230      1
89       1
906      1
Name: status, Length: 750, dtype: object

In [80]:
model = MLPRegressor(hidden_layer_sizes = 20, activation = "logistic", solver = "sgd", verbose = True)
model.fit(X_train, y_train)

Iteration 1, loss = 0.17403733
Iteration 2, loss = 0.15058901
Iteration 3, loss = 0.12122181
Iteration 4, loss = 0.09377920
Iteration 5, loss = 0.07236313
Iteration 6, loss = 0.05818907
Iteration 7, loss = 0.04915516
Iteration 8, loss = 0.04485966
Iteration 9, loss = 0.04284355
Iteration 10, loss = 0.04217836
Iteration 11, loss = 0.04212306
Iteration 12, loss = 0.04225094
Iteration 13, loss = 0.04231855
Iteration 14, loss = 0.04237034
Iteration 15, loss = 0.04234359
Iteration 16, loss = 0.04228582
Iteration 17, loss = 0.04221473
Iteration 18, loss = 0.04215943
Iteration 19, loss = 0.04213715
Iteration 20, loss = 0.04208811
Iteration 21, loss = 0.04207387
Training loss did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.


MLPRegressor(activation='logistic', hidden_layer_sizes=20, solver='sgd',
             verbose=True)

In [81]:
y_train

502      1
645    0.5
332      1
689      1
139    0.5
      ... 
389      0
119      1
230      1
89       1
906      1
Name: status, Length: 750, dtype: object

In [82]:
set(X_train['gender'].tolist())

{0.0, 1.0}

In [83]:
y_predicted = model.predict(X_test)

In [84]:
min(y_predicted)

0.8235463254077046

In [85]:
max(y_predicted)

0.8748161866812587

In [86]:
y_predicted

array([0.85307491, 0.86191624, 0.86146267, 0.85520504, 0.85472032,
       0.85217657, 0.86139324, 0.85532454, 0.86045817, 0.86078656,
       0.85187683, 0.85528881, 0.85633487, 0.85608673, 0.85985383,
       0.85239727, 0.85562241, 0.82354633, 0.8562583 , 0.85570751,
       0.85428656, 0.85510907, 0.85108167, 0.84287858, 0.85347435,
       0.86246841, 0.85446571, 0.85853017, 0.85437501, 0.85448499,
       0.84988696, 0.86073637, 0.85869369, 0.8555171 , 0.86089283,
       0.86072443, 0.85350268, 0.86313065, 0.86576182, 0.85963576,
       0.85483906, 0.86162017, 0.85450293, 0.86064552, 0.85372917,
       0.8539569 , 0.85136996, 0.85502289, 0.85395066, 0.85343316,
       0.86189446, 0.86237385, 0.85248467, 0.85387183, 0.86041096,
       0.85431838, 0.86734479, 0.8553995 , 0.85359347, 0.85339514,
       0.85952993, 0.85637892, 0.85597653, 0.86186041, 0.83831594,
       0.85043405, 0.85379457, 0.86264629, 0.8684155 , 0.82633503,
       0.8533893 , 0.84952886, 0.86110658, 0.8540988 , 0.85697

In [71]:
for i in range(len(y_predicted)):
    if y_predicted[i] <= 0.25:
        y = 0
    elif y_predicted[i] >= 0.75:
        y = 1
    else:
        y = 0.5
    y_predicted[i] = y


In [72]:
cm = confusion_matrix(y_test, y_predicted)
print(cm)

ValueError: Classification metrics can't handle a mix of unknown and binary targets

In [73]:
y_predicted

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1.

In [58]:
for i in range(100):
    print(y_predicted[i], y_test.iloc[i])

1.0 1
1.0 0.5
1.0 0.5
1.0 1
1.0 1
1.0 1
1.0 0.5
1.0 1
1.0 1
1.0 1
1.0 1
1.0 0.5
1.0 1
1.0 1
1.0 0.5
1.0 1
1.0 1
1.0 1
1.0 1
1.0 1
1.0 1
1.0 1
1.0 1
1.0 1
1.0 1
1.0 0.5
1.0 0
1.0 1
1.0 1
1.0 1
1.0 1
1.0 1
1.0 1
1.0 1
1.0 0
1.0 0.5
1.0 1
1.0 1
1.0 0
1.0 1
1.0 1
1.0 1
1.0 1
1.0 0
1.0 1
1.0 1
1.0 1
1.0 1
1.0 0.5
1.0 1
1.0 1
1.0 0
1.0 1
1.0 0
1.0 1
1.0 1
1.0 0.5
1.0 1
1.0 1
1.0 1
1.0 1
1.0 1
1.0 1
1.0 1
1.0 0.5
1.0 0.5
1.0 1
1.0 1
1.0 1
1.0 1
1.0 1
1.0 1
1.0 1
1.0 1
1.0 1
1.0 1
1.0 0.5
1.0 1
1.0 1
1.0 1
1.0 0.5
1.0 1
1.0 1
1.0 1
1.0 1
1.0 1
1.0 0.5
1.0 1
1.0 1
1.0 0.5
1.0 1
1.0 0.5
1.0 1
1.0 1
1.0 1
1.0 1
1.0 1
1.0 1
1.0 1
1.0 1


In [74]:
y_test

648      1
676    0.5
854    0.5
729      1
887      1
      ... 
331      1
949      1
890      1
7        1
786      1
Name: status, Length: 250, dtype: object

In [36]:
y_test.iloc[0]

1